# 机器学习模型对比

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
from skimage import measure

## 数据准备

In [ ]:
# 分别在不同模型下都运行，在config.py中更换模型
!python test.py -st 20160901 -et 20160928 -save

In [ ]:
# change the corresponding paths to your local paths
land_mask = np.load("data/land_mask.npy")
targets = np.load("test_results/targets.npy")
times = np.load("test_results/times.npy")
sic_pred_ConvLSTM = np.load("test_results/sic_pred_ConvLSTM.npy")
sic_pred_PredRNN = np.load("test_results/sic_pred_PredRNN.npy")
sic_pred_FCNet = np.load("test_results/sic_pred_FCNet.npy")

In [ ]:
pred_length = times.shape[0] // 2
pred_times = times[pred_length:]

pred_start_time = pred_times[0]
pred_end_time = pred_times[-1]

In [ ]:
pred_times

In [ ]:
sic_pred_ConvLSTM = sic_pred_ConvLSTM[0, :, 0, :]
sic_pred_PredRNN = sic_pred_PredRNN[0, :, 0, :]
sic_pred_FCNet = sic_pred_FCNet[0, :, 0, :]

In [ ]:
sic_pred_ConvLSTM.shape, sic_pred_PredRNN.shape, sic_pred_FCNet.shape, targets.shape, land_mask.shape, times.shape

## 绘制结果的函数

In [2]:
# Function to format tick labels as percentages
def percentage(x, pos):
    return f"{x:.0%}"

In [ ]:
# Function to plot sea ice concentration
def plot_sic(sic, cmap, save_path):
    os.makedirs(save_path, exist_ok=True)
    # 绘制海冰浓度图
    for i, time in enumerate(pred_times):
        fig, ax = plt.subplots()
        img = ax.imshow(sic[i], cmap=cmap, vmin=0, vmax=1)

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        # 显示colorbar
        # cbar = plt.colorbar(img)
        # cbar.ax.yaxis.set_major_formatter(FuncFormatter(percentage))

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

In [ ]:
# Function to plot sea ice concentration difference
def plot_diff(sic, cmap, save_path):
    os.makedirs(save_path, exist_ok=True)

    # 绘制海冰浓度图
    for i, time in enumerate(pred_times):
        fig, ax = plt.subplots()
        img = ax.imshow(sic[i], cmap=cmap, vmin=-1, vmax=1)

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        # 显示colorbar
        # cbar = plt.colorbar(img)
        # cbar.ax.yaxis.set_major_formatter(FuncFormatter(percentage))

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

In [ ]:
# 绘制SIE的函数
def plot_SIE(
    sic_pred_FCNet,
    sic_pred_PredRNN,
    sic_pred_ConvLSTM,
    targets,
    save_path,
):
    os.makedirs(save_path, exist_ok=True)

    # 绘制海冰SIE图
    for i, time in enumerate(pred_times):

        # 找到浓度大于0.15的部分
        thresholded_image_FCNet = sic_pred_FCNet[i] > 0.15
        thresholded_image_PredRNN = sic_pred_PredRNN[i] > 0.15
        thresholded_image_ConvLSTM = sic_pred_ConvLSTM[i] > 0.15
        thresholded_image_targets = targets[i] > 0.15

        # 计算边缘线
        contours_FCNet = measure.find_contours(thresholded_image_FCNet, 0.5)
        contours_PredRNN = measure.find_contours(thresholded_image_PredRNN, 0.5)
        contours_ConvLSTM = measure.find_contours(thresholded_image_ConvLSTM, 0.5)
        contours_targets = measure.find_contours(thresholded_image_targets, 0.5)

        # 创建新图
        fig, ax = plt.subplots()

        for contour in contours_FCNet:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#0000FF"
            )  # 指定线条颜色为蓝色

        for contour in contours_PredRNN:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#FF0000"
            )  # 指定线条颜色为红色

        for contour in contours_ConvLSTM:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#00A000"
            )  # 指定线条颜色为绿色

        for contour in contours_targets:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#000000"
            )  # 指定线条颜色为黑色

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

## 绘制SIC

In [ ]:
# 创建渐变色的colormap
colors = [(0, "#04629a"), (1, "white")]  # 定义颜色映射
cmap_ice_conc = LinearSegmentedColormap.from_list("cmap_ice_conc", colors)

In [ ]:
save_path = f"analysis_result/prediction/ConvLSTM/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_ConvLSTM, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/prediction/PredRNN/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_PredRNN, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/prediction/FCNet/{pred_start_time}-{pred_end_time}"
plot_sic(sic_pred_FCNet, cmap_ice_conc, save_path)

In [ ]:
save_path = f"analysis_result/ground_truth/{pred_start_time}-{pred_end_time}"
plot_sic(targets, cmap_ice_conc, save_path)

## 绘制SIC Difference

In [ ]:
# 创建自定义colormap
colors = [
    (0, "blue"),
    (0.5, "white"),
    (1, "red"),
]  # 透明色用 (1, 1, 1, 0) 表示
cmap_diff = LinearSegmentedColormap.from_list("cmap_diff", colors)

In [ ]:
diff_ConvLSTM = sic_pred_ConvLSTM - targets
save_path = f"analysis_result/difference/ConvLSTM/{pred_start_time}-{pred_end_time}"
plot_diff(diff_ConvLSTM, cmap_diff, save_path)

In [ ]:
diff_PredRNN = sic_pred_PredRNN - targets
save_path = f"analysis_result/difference/PredRNN/{pred_start_time}-{pred_end_time}"
plot_diff(diff_PredRNN, cmap_diff, save_path)

In [ ]:
diff_FCNet = sic_pred_FCNet - targets
save_path = f"analysis_result/difference/FCNet/{pred_start_time}-{pred_end_time}"
plot_diff(diff_FCNet, cmap_diff, save_path)

In [ ]:
diff_ConvLSTM_FCNet = np.abs(diff_ConvLSTM) - np.abs(diff_FCNet)
save_path = f"analysis_result/difference/sic_pred_ConvLSTM - sic_pred_FCNet/{pred_start_time}-{pred_end_time}"
plot_diff(diff_ConvLSTM_FCNet, cmap_diff, save_path)

In [ ]:
diff_PredRNN_FCNet = np.abs(diff_PredRNN) - np.abs(diff_FCNet)
save_path = f"analysis_result/difference/sic_pred_PredRNN - sic_pred_FCNet/{pred_start_time}-{pred_end_time}"
plot_diff(diff_PredRNN_FCNet, cmap_diff, save_path)

## 绘制SIE

In [ ]:
save_path = f"analysis_result/SIE/{pred_start_time}-{pred_end_time}"
plot_SIE(sic_pred_FCNet, sic_pred_PredRNN, sic_pred_ConvLSTM, targets, save_path)

# 和数值模型SEAS5的对比

In [2]:
import cdsapi
import os
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

## 数据准备

In [3]:
DATADIR = "/data2/SEAS5"

In [4]:
dataset = "seasonal-original-single-levels"
request = {
    "originating_centre": "ecmwf",
    "system": "5",
    "variable": ["sea_ice_cover"],
    # "year": ["2016", "2017", "2018", "2019", "2020"],
    "year": ["2007", "2012", "2019", "2020"],
    # "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"],
    "month": ["09"],
    "day": ["01"],
    "leadtime_hour": [
        "24",
        "48",
        "72",
        "96",
        "120",
        "144",
        "168",
        "192",
        "216",
        "240",
        "264",
        "288",
        "312",
        "336",
        "360",
        "384",
        "408",
        "432",
        "456",
        "480",
        "504",
        "528",
        "552",
        "576",
        "600",
        "624",
        "648",
        "672",
        "696",
        "720",
        "744",
        "768",
        "792",
        "816",
        "840",
        "864",
        "888",
        "912",
        "936",
        "960",
        "984",
        "1008",
    ],
    "data_format": "netcdf",
}

filename = f"{DATADIR}/seas5_daily_siconc.nc"

if not os.path.exists(filename):
    client = cdsapi.Client()
    client.retrieve(dataset, request, filename)
else:
    print(f"{filename} already exists, skipping download.")

2025-02-26 23:19:50,027 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-26 23:19:50,028 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-26 23:19:51,566 INFO Request ID is 3277657c-de7b-4a71-b644-ebaff7d3fc2f
2025-02-26 23:19:51,774 INFO status has been updated to accepted
2025-02-26 23:20:00,833 INFO status has been updated to running
2025-02-26 23:28:14,035 INFO status has been updated to successful


In [5]:
siconc = f"{DATADIR}/seas5_daily_siconc.nc"

In [6]:
# 使用 dask 打开数据集
ds = xr.open_dataset(siconc)  # 不指定分块方式

In [7]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:                  (number: 51, forecast_reference_time: 4,
                              forecast_period: 42, latitude: 181, longitude: 360)
Coordinates:
  * number                   (number) int64 408B 0 1 2 3 4 5 ... 46 47 48 49 50
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 200...
  * forecast_period          (forecast_period) timedelta64[ns] 336B 1 days .....
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] 1kB ...
Data variables:
    siconc                   (number, forecast_reference_time, forecast_period, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-26T15:26 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
# Create Xarray Data Array
da = ds["siconc"]

In [9]:
da

<xarray.DataArray 'siconc' (number: 51, forecast_reference_time: 4,
                            forecast_period: 42, latitude: 181, longitude: 360)> Size: 2GB
[558290880 values with dtype=float32]
Coordinates:
  * number                   (number) int64 408B 0 1 2 3 4 5 ... 46 47 48 49 50
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 200...
  * forecast_period          (forecast_period) timedelta64[ns] 336B 1 days .....
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               (forecast_reference_time, forecast_period) datetime64[ns] 1kB ...
Attributes: (12/33)
    GRIB_paramId:                             31
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      65160
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               (0 - 1)
    long_name:                                Sea ice area fraction
    units:                                    (0 - 1)
    standard_name:                            sea_ice_area_fraction
    GRIB_surface:                             0.0

In [10]:
# 计算平均值
da_mean = da.mean(dim="number", keep_attrs=True)

In [11]:
forecast_reference_time = da_mean.coords["forecast_reference_time"].to_index()

In [12]:
forecast_reference_time

DatetimeIndex(['2007-09-01', '2012-09-01', '2019-09-01', '2020-09-01'], dtype='datetime64[ns]', name='forecast_reference_time', freq=None)

In [13]:
forecast_period = da_mean.coords["forecast_period"].to_index()

In [14]:
forecast_period

TimedeltaIndex([ '1 days',  '2 days',  '3 days',  '4 days',  '5 days',
                 '6 days',  '7 days',  '8 days',  '9 days', '10 days',
                '11 days', '12 days', '13 days', '14 days', '15 days',
                '16 days', '17 days', '18 days', '19 days', '20 days',
                '21 days', '22 days', '23 days', '24 days', '25 days',
                '26 days', '27 days', '28 days', '29 days', '30 days',
                '31 days', '32 days', '33 days', '34 days', '35 days',
                '36 days', '37 days', '38 days', '39 days', '40 days',
                '41 days', '42 days'],
               dtype='timedelta64[ns]', name='forecast_period', freq=None)

## 投影坐标转换

In [16]:
Max_SIE = 25889.0

def BACC_func(pred, true, mask):
    # 使用布尔索引将大于等于0.15的位置设置为1，其他地方设置为0
    pred_binary = (pred >= 0.15).astype(np.float32)
    true_binary = (true >= 0.15).astype(np.float32)

    # 计算 IIEE
    IIEE = np.sum(np.abs(pred_binary - true_binary) * mask)

    # 计算 BACC
    BACC = 1 - IIEE / Max_SIE
    return BACC

In [17]:
AMAP_mask = np.load("data/AMAP_mask.npy")

# Helper function to process each date independently
def process_date(date):
    BACC_results = {}
    pbar = tqdm(forecast_period)
    for lead_time in pbar:
        pbar.set_description(f"Processing lead time {lead_time} from {date}")
        osi_file = f"/data2/OSI-SAF/{date.year}/{date.month:02d}/ice_conc_nh_ease2-250_cdr-v3p0_{date.strftime('%Y%m%d')}1200.nc"
        osi_data = xr.open_dataset(osi_file)

        # Get lat, lon, and sea ice concentration data from the OSI file
        osi_lat = osi_data["lat"].values
        osi_lon = osi_data["lon"].values
        osi_sic = osi_data["ice_conc"].values[0]

        # Read seas5 data
        seas5_data = da_mean.sel(
            forecast_reference_time=date, forecast_period=lead_time
        )
        seas5_lat = seas5_data["latitude"].values
        seas5_lon = seas5_data["longitude"].values
        seas5_sic = seas5_data.values

        # Initialize result_sic
        result_sic = np.where(np.isnan(osi_sic), np.nan, 0)

        # Loop through osi_lat and osi_lon
        for i in range(osi_lat.shape[0]):
            for j in range(osi_lon.shape[1]):
                if np.isnan(result_sic[i, j]):
                    continue

                # Find closest indices in seas5_lat and seas5_lon
                lat_diff = np.abs(seas5_lat - osi_lat[i, j])
                lon_diff = np.abs((seas5_lon + 180) % 360 - 180 - osi_lon[i, j])
                lat_idx = np.argmin(lat_diff)
                lon_idx = np.argmin(lon_diff)

                # Update result_sic if indices are valid
                if lat_idx < seas5_lat.shape[0] and lon_idx < seas5_lon.shape[0]:
                    result_sic[i, j] = seas5_sic[lat_idx, lon_idx]

        # Compute BACC
        BACC = BACC_func(result_sic, osi_sic, AMAP_mask)
        BACC_results[lead_time] = BACC

    return date, BACC_results


# Dictionary to store BACC results
BACC_dict = {}

# Use ProcessPoolExecutor for parallel processing
with ProcessPoolExecutor() as executor:
    # Submit tasks for each date
    futures = {
        executor.submit(process_date, date): date
        for date in forecast_reference_time
    }

    # Collect results as they complete
    for future in futures:
        date, BACC_results = future.result()
        BACC_dict[date] = BACC_results

Processing lead time 42 days 00:00:00 from 2012-09-01 00:00:00: 100%|██████████| 42/42 [01:02<00:00,  1.49s/it]
Processing lead time 42 days 00:00:00 from 2020-09-01 00:00:00: 100%|██████████| 42/42 [01:02<00:00,  1.49s/it]
Processing lead time 42 days 00:00:00 from 2019-09-01 00:00:00: 100%|██████████| 42/42 [01:03<00:00,  1.50s/it]
Processing lead time 42 days 00:00:00 from 2007-09-01 00:00:00: 100%|██████████| 42/42 [01:09<00:00,  1.65s/it]


In [19]:
BACC_dict

{Timestamp('2007-09-01 00:00:00'): {Timedelta('1 days 00:00:00'): np.float32(0.9291205),
  Timedelta('2 days 00:00:00'): np.float32(0.92803895),
  Timedelta('3 days 00:00:00'): np.float32(0.92514193),
  Timedelta('4 days 00:00:00'): np.float32(0.92205185),
  Timedelta('5 days 00:00:00'): np.float32(0.920584),
  Timedelta('6 days 00:00:00'): np.float32(0.9194639),
  Timedelta('7 days 00:00:00'): np.float32(0.91857547),
  Timedelta('8 days 00:00:00'): np.float32(0.9186527),
  Timedelta('9 days 00:00:00'): np.float32(0.91903895),
  Timedelta('10 days 00:00:00'): np.float32(0.9186527),
  Timedelta('11 days 00:00:00'): np.float32(0.91884583),
  Timedelta('12 days 00:00:00'): np.float32(0.9194639),
  Timedelta('13 days 00:00:00'): np.float32(0.91977286),
  Timedelta('14 days 00:00:00'): np.float32(0.91973424),
  Timedelta('15 days 00:00:00'): np.float32(0.9198501),
  Timedelta('16 days 00:00:00'): np.float32(0.92031366),
  Timedelta('17 days 00:00:00'): np.float32(0.92015916),
  Timedelta('1

In [ ]:
# 将字典中的数据写入 Excel 文件
df = pd.DataFrame(BACC_dict)
df.to_excel("test_results/BACC_SEAS5.xlsx")
print("BACC values saved to BACC_SEAS5.xlsx")

## 绘制SEAS5和FCNet的预测结果

In [23]:
# 分别在不同模型下都运行，在config.py中更换模型
!python test.py -st 20190831 -et 20190927 -save


2025-02-26 23:51:26
######################## Start testing! ########################

Model Configurations:

{'model': 'ConvLSTM', 'data_paths': 'data/data_path.txt', 'train_log_path': 'train_logs', 'test_results_path': 'test_results', 'batch_size_vali': 16, 'batch_size': 4, 'lr': 0.0005, 'weight_decay': 0.01, 'num_epochs': 200, 'early_stop': True, 'patience': 20, 'gradient_clip': True, 'clip_threshold': 1.0, 'num_workers': 32, 'img_size': (432, 432), 'input_dim': 1, 'output_dim': 1, 'input_length': 14, 'pred_length': 14, 'input_gap': 1, 'pred_gap': 1, 'pred_shift': 14, 'train_period': (19910101, 20100101), 'eval_period': (20100101, 20151231), 'kernel_size': (3, 3), 'patch_size': (2, 2), 'hidden_dim': (64, 64, 64, 64)}

Arguments:
  start time: 20190831
  end time: 20190927
  output_dir: test_results
  data_paths: data/data_path.txt
  save_result: True

Testing......

Metrics: mse: 0.00641, rmse: 0.08007, mae: 0.02633, nse: 0.92296, PSNR: 21.93091, BACC: 0.95289, loss: 0.00094

Saving

In [24]:
# 选择 forecast_reference_time 为 2019-09-01 的数据
selected_data = da_mean.sel(forecast_reference_time="2019-09-01")

# 选择 forecast_period 为 15 至 28 天的数据
selected_period = selected_data.sel(forecast_period=slice("14 days", "27 days"))

In [25]:
selected_period

<xarray.DataArray 'siconc' (forecast_period: 14, latitude: 181, longitude: 360)> Size: 4MB
array([[[0.8145342 , 0.8145342 , 0.8145342 , ..., 0.8145342 ,
         0.8145342 , 0.8145342 ],
        [0.8406408 , 0.8397408 , 0.83883846, ..., 0.84362435,
         0.8426277 , 0.8416356 ],
        [0.8830626 , 0.88115406, 0.8792449 , ..., 0.88802475,
         0.8863711 , 0.88471806],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.8208615 , 0.8208615 , 0.8208615 , ..., 0.8208615 ,
         0.8208615 , 0.8208615 ],
        [0.849814  , 0.8489673 , 0.8481197 , ..., 0.8523377 ,
         0.8514964 , 0.850656  ],
        [0.8891006 , 0.8872636 , 0.8854283 , ..., 0.89377576,
         0.8922155 , 0.8906582 ],
...
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.8828897 , 0.8828897 , 0.8828897 , ..., 0.8828897 ,
         0.8828897 , 0.8828897 ],
        [0.9081747 , 0.9076511 , 0.9071263 , ..., 0.91038156,
         0.90964526, 0.9089104 ],
        [0.92322105, 0.9220719 , 0.9209242 , ..., 0.92648494,
         0.9253977 , 0.92430925],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2019-09-01
  * forecast_period          (forecast_period) timedelta64[ns] 112B 14 days ....
  * latitude                 (latitude) float64 1kB 90.0 89.0 ... -89.0 -90.0
  * longitude                (longitude) float64 3kB 0.0 1.0 2.0 ... 358.0 359.0
    valid_time               (forecast_period) datetime64[ns] 112B ...
Attributes: (12/33)
    GRIB_paramId:                             31
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      65160
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               (0 - 1)
    long_name:                                Sea ice area fraction
    units:                                    (0 - 1)
    standard_name:                            sea_ice_area_fraction
    GRIB_surface:                             0.0

In [26]:
forecast_reference_time = selected_period.coords["forecast_reference_time"].values
forecast_period = selected_period.coords["forecast_period"].to_index()

In [27]:
# 将 numpy.datetime64 对象转换为 pandas.Timestamp 对象
date = pd.Timestamp(forecast_reference_time)

In [28]:
# Initialize an empty list to store result_sic
sic_pred_SEAS5 = []

pbar = tqdm(forecast_period)
for lead_time in pbar:
    pbar.set_description(f"Processing lead time {lead_time}")
    osi_file = f"/data2/OSI-SAF/{date.year}/{date.month:02d}/ice_conc_nh_ease2-250_cdr-v3p0_{date.strftime('%Y%m%d')}1200.nc"
    osi_data = xr.open_dataset(osi_file)

    # Get lat, lon, and sea ice concentration data from the OSI file
    osi_lat = osi_data["lat"].values
    osi_lon = osi_data["lon"].values
    osi_sic = osi_data["ice_conc"].values[0]

    # Read seas5 data
    seas5_data = da_mean.sel(forecast_reference_time=date, forecast_period=lead_time)
    seas5_lat = seas5_data["latitude"].values
    seas5_lon = seas5_data["longitude"].values
    seas5_sic = seas5_data.values

    # Initialize result_sic
    result_sic = np.where(np.isnan(osi_sic), np.nan, 0)

    # Loop through osi_lat and osi_lon
    for i in range(osi_lat.shape[0]):
        for j in range(osi_lon.shape[1]):
            if np.isnan(result_sic[i, j]):
                continue

            # Find closest indices in seas5_lat and seas5_lon
            lat_diff = np.abs(seas5_lat - osi_lat[i, j])
            lon_diff = np.abs((seas5_lon + 180) % 360 - 180 - osi_lon[i, j])
            lat_idx = np.argmin(lat_diff)
            lon_idx = np.argmin(lon_diff)

            # Update result_sic if indices are valid
            if lat_idx < seas5_lat.shape[0] and lon_idx < seas5_lon.shape[0]:
                result_sic[i, j] = seas5_sic[lat_idx, lon_idx]

    # Append result_sic to the list
    sic_pred_SEAS5.append(result_sic)

# Stack all result_sic arrays along a new dimension
sic_pred_SEAS5 = np.stack(sic_pred_SEAS5, axis=0)

Processing lead time 27 days 00:00:00: 100%|██████████| 14/14 [00:21<00:00,  1.56s/it]


In [29]:
# change the corresponding paths to your local paths
land_mask = np.load("data/land_mask.npy")
targets = np.load("test_results/targets.npy")
times = np.load("test_results/times.npy")
sic_pred_ConvLSTM = np.load("test_results/sic_pred_ConvLSTM.npy")
sic_pred_PredRNN = np.load("test_results/sic_pred_PredRNN.npy")
sic_pred_FCNet = np.load("test_results/sic_pred_FCNet.npy")

In [30]:
sic_pred_FCNet = sic_pred_FCNet[0, :, 0, :]
sic_pred_ConvLSTM = sic_pred_ConvLSTM[0, :, 0, :]
sic_pred_PredRNN = sic_pred_PredRNN[0, :, 0, :]

In [31]:
pred_length = times.shape[0] // 2
pred_times = times[pred_length:]

pred_start_time = pred_times[0]
pred_end_time = pred_times[-1]

In [32]:
pred_times

array([20190914, 20190915, 20190916, 20190917, 20190918, 20190919,
       20190920, 20190921, 20190922, 20190923, 20190924, 20190925,
       20190926, 20190927])

In [33]:
sic_pred_SEAS5.shape, sic_pred_ConvLSTM.shape, sic_pred_PredRNN.shape, sic_pred_FCNet.shape, targets.shape, land_mask.shape, times.shape

((14, 432, 432),
 (14, 432, 432),
 (14, 432, 432),
 (14, 432, 432),
 (14, 432, 432),
 (432, 432),
 (28,))

In [34]:
# 绘制SIE的函数
def plot_SIE(
    sic_pred_FCNet,
    sic_pred_PredRNN,
    sic_pred_ConvLSTM,
    sic_pred_SEAS5,
    targets,
    save_path,
):
    os.makedirs(save_path, exist_ok=True)

    # 绘制海冰SIE图
    for i, time in enumerate(pred_times):

        # 找到浓度大于0.15的部分
        thresholded_image_FCNet = sic_pred_FCNet[i] > 0.15
        thresholded_image_PredRNN = sic_pred_PredRNN[i] > 0.15
        thresholded_image_ConvLSTM = sic_pred_ConvLSTM[i] > 0.15
        thresholded_image_SEAS5 = sic_pred_SEAS5[i] > 0.15
        thresholded_image_targets = targets[i] > 0.15

        # 计算边缘线
        contours_FCNet = measure.find_contours(thresholded_image_FCNet, 0.5)
        contours_PredRNN = measure.find_contours(thresholded_image_PredRNN, 0.5)
        contours_ConvLSTM = measure.find_contours(thresholded_image_ConvLSTM, 0.5)
        contours_SEAS5 = measure.find_contours(thresholded_image_SEAS5, 0.5)
        contours_targets = measure.find_contours(thresholded_image_targets, 0.5)

        # 创建新图
        fig, ax = plt.subplots()

        for contour in contours_FCNet:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#0000FF"
            )  # 指定线条颜色为蓝色

        for contour in contours_PredRNN:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#FF0000"
            )  # 指定线条颜色为红色

        for contour in contours_ConvLSTM:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#00A000"
            )  # 指定线条颜色为绿色

        for contour in contours_SEAS5:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#CC6CE7"
            )  # 指定线条颜色为紫色

        for contour in contours_targets:
            ax.plot(
                contour[:, 1], contour[:, 0], linewidth=0.7, color="#000000"
            )  # 指定线条颜色为黑色

        # 绘制陆地
        land_color = "#d2b48c"  # 陆地的颜色
        land = np.ma.masked_where(land_mask == False, land_mask)  # 创建陆地的掩模
        ax.imshow(land, cmap=ListedColormap([land_color]))

        # 关闭坐标轴和标签
        ax.set_title("")  # 禁用标题
        ax.set_xlabel("")  # 禁用 x 轴标签
        ax.set_ylabel("")  # 禁用 y 轴标签
        ax.set_xticks([])  # 禁用 x 轴刻度
        ax.set_yticks([])  # 禁用 y 轴刻度

        plt.savefig(f"{save_path}/{time}.png", bbox_inches="tight")

        plt.close()

In [35]:
save_path = f"analysis_result/SIE/SEAS5-{pred_start_time}-{pred_end_time}"
plot_SIE(
    sic_pred_FCNet,
    sic_pred_PredRNN,
    sic_pred_ConvLSTM,
    sic_pred_SEAS5,
    targets,
    save_path,
)